In [1]:
import sys
import os

sys.path.append(os.path.abspath("../../data/API_KEY"))

In [24]:
import openai
import OpenAI_API_Key

API_KEY = OpenAI_API_Key.API_KEY

os.environ["OPENAI_API_KEY"] = API_KEY

openai.__version__

'1.51.2'

In [3]:
save_path = "../../data/nlp/2020_경제금융용어 700선_게시.pdf"
os.system(
    f'curl -o "{save_path}" https://raw.githubusercontent.com/chatgpt-kr/openai-api-tutorial/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf'
)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5806k  100 5806k    0     0  1536k      0  0:00:03  0:00:03 --:--:-- 1536k


0

In [4]:
import matplotlib.pyplot as plt
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 데이터 확인

In [5]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(save_path)
texts = loader.load_and_split()

print(len(texts))

366


In [6]:
texts[15]

Document(metadata={'source': '../../data/nlp/2020_경제금융용어 700선_게시.pdf', 'page': 19}, page_content='3ㄱ \n신용을 모두 포괄하는 것으로 크게 가계대출과 판매신용으로 구분된다. 현재 가계신용통\n계는 2002년 말 잔액부터 분기별로 제공되고 있다. 가계신용통계는 우리나라 가계부문의 \n부채 규모 및 변동 등을 파악하는 데 널리 활용되고 있다 .\n가계처분가능소득 \n가계처분가능소득(PDI; Personal Disposable Income)은 가계가 맘대로 소비와 저축\n으로 처분할 수 있는 소득을 의미한다. 흔히 국민들의 생활수준을 파악해 볼 수 있는 \n지표로 1인당 GNI가 널리 쓰이고 있으나 국민총소득에는 가계 뿐 아니라 기업 금융기관 \n정부가 벌어 들인 소득이 모두 포함되어 있다. 따라서 기업과 금융기관 등이 가계부문 \n보다 더 많은 소득을 벌어 1인당 국민총소득(GNI)이 높아진 경우에는 가계가 느끼는 \n체감경기는 전체 경기와 괴리가 있게 된다. 1인당 가계총처분가능소득(PGDI; Personal \nGross Disposable Income)은 가계부문의 총처분가능소득을 연앙인구로 나누어 계산한 \n지표로 가계의 구매력을 가장 정확히 가늠해 볼 수 있는 소득지표이다 . \n 연관검색어 : 국민총소득 (GNI)\n가교은행\n청산 대상 금융기관의 자산, 부채를 임시로 넘겨받아 예금, 출금 등의 업무를 대행하고 \n합병, 채권채무관계 조정 등 후속조치를 수행하는 은행을 말한다. 금융기관이 파산한 경우 \n예금보험제도는 청산, 매각, 자산부채승계, 가교은행을 통한 인수 등의 수단을 통해 부실 \n금융기관을 처리한다. 이 중 가교은행을 통한 인수방식은 파산은행의 처리에 상당한 시간이 \n소요될 것으로 예상할 경우 사용되며 새로운 은행을 설립하여 자산, 부채를 포괄승계하도록 \n하고, 인수 희망자를 물색하는 등의 조치를 취하게 된다. 가교은행을 이용한 처리방식은 \n금융기관 파산에

In [7]:
texts[0].page_content

'iii찾아보기\n한국은행은 국민들이 경제 및 금융에 대한 이해도를 높이고 경제에 관한 합리적인 \n의사결정 능력을 키울 수 있도록 현장 경제교육, 온라인 경제교육, 경제교육 콘텐츠 \n개발 등 대국민 경제교육을 다양하게 수행해 오고 있습니다 . \n이의 일환으로 2018년 \n 경제금융용어 700선\n 을 발간하였는데 그간 동 책자에 대한 \n수요가 꾸준히 늘어남에 따라 이번에 추가로 발간하게 되었습니다 .\n지난번 내용과 같이 통화정책, 실물경제, 금융안정, 지급결제 등 한국은행 주요 \n업무를 이해하는데 필요한 전문 용어와 경제·금융 흐름을 이해하는데 도움이 되는 \n시사 경제금융 용어들을 수록하였습니다. 용어해설은 개념과 도입 배경, 의미, 적용 \n사례 등을 담아 쉽게 이해할 수 있도록 하였습니다. 또한 e-book으로도 제작하여 \n독자들의 편의성과 가독성을 높였습니다 . \n코로나 19로 이전에 경험하지 못한 경제위기를 겪으면서 경제·금융 현상에 대한 \n일반인들의 관심도가 이전에 비해 높아졌습니다. 아무쪼록 이 책자가 한국은행의 주요 \n정책과 국내외 경제를 이해하는 데 조금이나마 도움이 되기를 바라겠습니다. 끝으로 \n경제금융용어 700선\n  집필에 참여해주신 경제교육실 교수님들과 용어 집필에 좋은 \n의견을 주신 관련부서 여러분께 감사드립니다 .\n2020년 8월\n한국은행 경제교육실장 박 철 원머 리 말'

In [8]:
texts[5].page_content

'vi경제금융용어 700선\nㅂ\n바이오인증 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ 118\n바젤은행감독위원회 /\n바젤위원회(BCBS) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ 118\n발행시장 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・ 119\n발행중지화폐/유통정지화폐 ・・・・・・・・・・・・・・・ 120\n방카슈랑스 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ 120\n배당할인모형 ・ ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・ 121\n밴드웨건효과 ・ ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・ 121\n뱅 크 런 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・ 122\n범위의 경제 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ 122\n법률리스크 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ 122\n베블런효과 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ 123\n변동금리 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・ 123\n변동금리부채권(FRN) ・ ・ ・ ・・・・・・・・・・・・・・・・・・・・・・ 123\n보기화폐(견양화폐) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ 

In [9]:
texts[13].page_content

"1ㄱ \nㄱ\n가계부실위험지수(HDRI)\n가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 \n부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상\n환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt \nTo Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA 가 \n각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 \n가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’,  \n자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다 . \n다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 \n이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .\n 연관검색어 : 총부채원리금상환비율(DSR)\n가계수지\n가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 \n표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지 가 가계수지 ∙"

In [10]:
# 머릿말, 목차는 필요 없으니까(금융 용어에 대한 내용만 있으면 되니까) 제거
texts = texts[13:]
texts[0]

Document(metadata={'source': '../../data/nlp/2020_경제금융용어 700선_게시.pdf', 'page': 17}, page_content="1ㄱ \nㄱ\n가계부실위험지수(HDRI)\n가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 \n부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상\n환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt \nTo Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA 가 \n각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 \n가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’,  \n자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다 . \n다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 \n이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .\n 연관검색어 : 총부채원리금상환비율(DSR)\n가계수지\n가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 \n표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지 가 가계수지 ∙")

In [11]:
texts[-1]

Document(metadata={'source': '../../data/nlp/2020_경제금융용어 700선_게시.pdf', 'page': 369}, page_content='경제금융용어 700선\n발행인 이주열\n편집인 박철원\n발행처 한국은행(www.bok.or.kr)\n서울특별시 중구 세종대로 67(태평로 2가)\n발행일 2020년 8월 3일\n인  쇄 ㈜제일프린테크 \n 본 자료는 한국은행 홈페이지(http://www.bok.or.kr>경제교육>온라인학습>일반인) 에서 \n다운로드 받으실 수 있습니다.\n 이 책자에 대한 문의는 한국은행 경제교육실 경제교육기획팀(02-759-5618)으로 연락 \n하여 주시기 바랍니다.\n \n경제금융용어 700선\n 은 \n① 정부간행물판매센터(☎ 02-734-6818, http://www.gpcb ooks.co.kr) \n② 한국경제서적(☎ 02-737-7498)③ 경제서적(☎ 02-736-0640, http://kj-book.co.kr) 또는 주요 서점 등에서 매권당 \n8,000원에 판매하고 있습니다.\nISBN 979-11-5538-393-3 03320')

In [12]:
# 맺음말 부분도 제거
texts = texts[:-1]

# Vector Store 정의

In [13]:
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts, embedding=embedding)

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_28331/3709482107.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


### _collection
- 벡터 스토어제 저장된 실제 데이터에 접근하기 위한 방법

In [14]:
# vectordb에 저장된 문서의 개수 확인
vectordb._collection.count()

352

In [15]:
# 사용 가능한 키 확인
vectordb._collection.get().keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data', 'included'])

In [16]:
# 문서 로딩
documents = vectordb._collection.get()["documents"]
documents[0]

"1ㄱ \nㄱ\n가계부실위험지수(HDRI)\n가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 \n부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상\n환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt \nTo Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA 가 \n각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 \n가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’,  \n자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다 . \n다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 \n이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .\n 연관검색어 : 총부채원리금상환비율(DSR)\n가계수지\n가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 \n표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지 가 가계수지 ∙"

In [17]:
# 임베딩 벡터 확인
result = vectordb._collection.get()["embeddings"]
print(result)

None


In [18]:
result = vectordb._collection.get(include=["embeddings"])["embeddings"]
result[0]

array([-0.00693841, -0.00727077,  0.0133149 , ..., -0.01831056,
       -0.00755174, -0.00942254])

In [19]:
# 메타데이터 확인.
metadatas = vectordb._collection.get()["metadatas"]
metadatas[0]  # 문서의 출처

{'page': 17, 'source': '../../data/nlp/2020_경제금융용어 700선_게시.pdf'}

# Retriever 생성

In [20]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [22]:
docs = retriever.get_relevant_documents("비트코인에 대해 설명해줘")

print(docs[0])

print("=" * 100)

print(docs[1])

page_content='139ㅂ 
비트코인
비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템
은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디지털통화라고 불린다 . 
이는 사토시 나카모토라는 사람(집단)에 의해 만들어져서 2009년 개방형 소프트웨어로 
배포되었다. 이 시스템은 공유형(peer-to-peer)이며, 거래는 중개자 없이 블록체인 소프트
웨어를 이용하는 참여자(nodes) 사이에 직접 이뤄진다. 이런 거래들은 공유(P2P) 네트워크
상 참여자의 작업증명(proof-of-work)을 통해 검증되고 공개된 분산원장인 블록체인에 
기록된다. 승인된 거래의 새 그룹인 한 블록은 대략 10분에 한 번씩 만들어져서 블록체인에 
추가되고 신속하게 모든 참여자에게 보내어진다. 비트코인은 대규모 전기를 사용하는 
컴퓨터 처리 능력(power)을 활용해서 이뤄지는 기록보관 서비스인 채굴(mining)에 대한 
보상으로 발행되고 있으며 다른 통화 ･상품･용역 등과 교환되어가고 있다. 중앙은행이 
발행한 법정화폐가 아닌 비트코인은 비트코인 플랫폼에서 거래되며 투자대상으로서도 
관심을 받고 있다. 하지만 급등한 가격 및 심한 변동 폭으로 거품논란이 크다. 또한 익명성으
로 자금세탁 등 불법거래에 악용될 소지가 많아 중국 등 일부 국가에서 비트코인 등 가상통
화의 거래를 규제하고 있다. 일본의 경우 비트코인의 거래이익에 대해 세금을 부과한다 . 
비트코인은 추가되는 한 블록당 새로 12.5비트코인을 보상하는데(2016.7월 현재), 21 만개
가 채굴될 때(대략 4년)마다 그 보상이 반으로 줄어든다. 비트코인의 총량은 21 백만개이며 
2140년경 모두 채굴될 것으로 전망된다. 비트코인은 그 시스템의 설계시 그 수량을 한정시
켜 놓았기 때문에 원칙적으로 인플레이션에 의해 가치가 떨어질 수 있는 화폐와 다른 
속성을 지닌다. 한편 2017년 8월 1일 비트코인(classic bitcoin)에서 ‘비트코인캐시’(BC

# 챗봇 만들기 with RAG

이제 ChatGPT API와 이미 만들어진 Prompt를 통해서 간단히 챗봇을 구현해봅시다. RetrievalQA.from_chain_type()의 llm 매개변수의 값으로 초기에 임포트한 OpenAI()를 사용할 경우, 기본값으로 ChatGPT API를 사용합니다.chain_type의 매개변수의 값으로 "stuff"를 사용할 경우, 사용자의 눈에는 보이지 않지만 자동으로 아래의 프롬프트를 사용하여 챗봇을 구현합니다.  

해석해보면 주어진 질문과 본문을 통해서 답변을 하되, 만약 답변할 수 없다면 답변을 임의로 하지말고 모른다고 하라는 내용입니다.
```
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{텍스트}

{질문}
```

{텍스트}에는 사용자의 질문으로부터 높은 유사도를 가진 텍스트가 들어가게 되고, {질문} 부분은 사용자의 질문이 들어가게 됩니다. retriever는 입력된 사용자의 질문으로부터 유사도를 계산하는 도구를 구현하여 매개변수의 값으로 넘겨주면 됩니다. 앞서 이미 Chroma를 통해 벡터 도구 객체로부터 구현한 retreiver를 사용합니다.  

return_source_documents는 챗봇의 답변에 사용된 텍스트들의 출처를 표시할 것인지를 의미합니다. return_source_documents의 값을 False로 할 경우, 챗봇의 답변이 어떤 텍스트에 근거하였는지 알 수 없습니다. 여기서는 뒤에서 근거가 되는 텍스트의 출처를 확인할 것이므로 True를 사용합니다.

In [26]:
from langchain import PromptTemplate

# 챗봇이 어떻게 동작해야 할지에 대한 정보 제공
template = """당신은 한국경제신문에서 만든 금융 용어를 설명해주는 챗봇입니다.
주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면, 답변할 수 없다고 하세요.
존댓말로 정중하게 대답해주세요.
{context}

Question: {question}
Answer:
"""

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='당신은 한국경제신문에서 만든 금융 용어를 설명해주는 챗봇입니다.\n주어진 검색 결과를 바탕으로 답변하세요.\n검색 결과에 없는 내용이라면, 답변할 수 없다고 하세요.\n존댓말로 정중하게 대답해주세요.\n{context}\n\nQuestion: {question}\nAnswer:\n')

In [27]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    streaming=True,
    temperature=0,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [29]:
# 기존의 ChatGPT 의 prompt 를 개발자가 작성한 것으로 대체
# {context} 에 해당하는 조회 결과가 답변할 때 들어가도록 설정
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt": prompt},
    retriever=retriever,  # 검색기의 조회 결과를 {context} 에 삽입
    return_source_documents=True,
)

# 사용하기

In [31]:
query = "비트코인에 대해 설명해주세요."
chatbot_response = qa_chain(query)

비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템으로, 중앙 저장소나 단일 관리자가 없는 최초의 탈중앙화된 디지털 통화입니다. 비트코인은 사토시 나카모토라는 인물(또는 집단)에 의해 2009년에 개발되어 개방형 소프트웨어로 배포되었습니다.

비트코인 시스템은 공유형(peer-to-peer) 구조를 가지고 있으며, 거래는 중개자 없이 블록체인 소프트웨어를 이용하는 참여자(nodes) 간에 직접 이루어집니다. 이러한 거래는 참여자들의 작업증명(proof-of-work)을 통해 검증되며, 공개된 분산원장인 블록체인에 기록됩니다. 새로운 거래가 승인되면, 대략 10분마다 새로운 블록이 생성되어 블록체인에 추가되고 모든 참여자에게 전송됩니다.

비트코인은 채굴(mining)이라는 과정을 통해 발행되며, 이는 대규모 전기를 사용하는 컴퓨터 처리 능력을 활용하여 기록 보관 서비스를 제공합니다. 비트코인의 총량은 2100만 개로 한정되어 있으며, 2140년경에 모두 채굴될 것으로 예상됩니다. 비트코인은 법정화폐가 아니며, 비트코인 플랫폼에서 거래되며 투자 대상으로도 관심을 받고 있습니다. 그러나 가격의 급등과 심한 변동성으로 인해 거품 논란이 있으며, 익명성으로 인해 자금세탁 등 불법 거래에 악용될 소지가 있어 일부 국가에서는 규제를 시행하고 있습니다.

또한, 비트코인은 하드포크(hard-fork)를 통해 비트코인캐시(BCH)와 비트코인골드(BTG)와 같은 다른 암호화폐로 분리되기도 했습니다. 비트코인은 인플레이션에 대한 저항력을 가지고 있으며, 그 설계에 따라 수량이 한정되어 있습니다.

# Gradio 를 통한 Chatbot 배포

In [32]:
%pip install gradio==3.50.2

  Using cached altair-5.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached referencing-0.35.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.20.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 13.3 MB/s eta 0:00:00a 0:00:01
Using cached altair-5.4.1-py3-none-any.whl (658 kB)
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
Using cached referencing-0.35.1-py3-none-any.whl (26 kB)
Using cached rpds_py-0.20.0-cp311-cp311-macosx_11_0_arm64.whl (311 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 13.1
    Uninstalling websockets-13.1:
      Successfully uninstalled websockets-13.1
Note: you may need to restart the kernel to use updated packages.


In [33]:
import gradio as gr

# 인터페이스를 생성.
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        label="경제금융용어 챗봇"
    )  # 경제금융용어 챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창의 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
        result = qa_chain(message)
        bot_message = result["result"]

        # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
        chat_history.append((message, bot_message))
        return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행.
demo.launch(debug=True)

/opt/anaconda3/envs/dl-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.50.2, however version 5.0.1 is available, please upgrade.
--------
비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템으로, 중앙 저장소나 단일 관리자가 없는 최초의 탈중앙화된 디지털 통화입니다. 2009년 사토시 나카모토라는 인물(또는 집단)에 의해 만들어져 개방형 소프트웨어로 배포되었습니다. 비트코인은 공유형(peer-to-peer) 시스템을 기반으로 하며, 거래는 중개자 없이 참여자(nodes) 간에 직접 이루어집니다.

비트코인의 거래는 작업증명(proof-of-work) 방식으로 검증되며, 모든 거래는 공개된 분산원장인 블록체인에 기록됩니다. 새로운 거래가 승인되면 약 10분마다 새로운 블록이 생성되어 블록체인에 추가됩니다. 비트코인은 채굴(mining)이라는 과정을 통해 발행되며, 이 과정에서 대규모 전기를 사용하는 컴퓨터 처리 능력이 필요합니다. 현재 한 블록당 12.5 비트코인이 보상으로 주어지며, 약 4년마다 이 보상은 반으로 줄어듭니다. 비트코인의 총량은 2100만 개로 한정되어 있으며, 2140년경에 모두 채굴될 것으로 예상됩니다.

비트코인은 법정화폐가 아니며, 비트코인 플랫폼에서 거래됩니다. 투자 대상으로도 관심을 받고 있지만, 가격의 급등과 심한 변동성으로 인해 거품 논란이 존재합니다. 또한 익명성으로 인해 자금세탁 등 불법 거래에 악용될 가능성이 있어 일부 국가에서는 비트코인 거래를 규제하고 있습니다. 일본은 비트코인 거래 이익에 대해 세금을 부과하고 있습니다. 

비트코인은 2017년 8월 1일에 비트코인캐시(BCH)로 하드포크되었고, 10월 25일에는 비트코인골드(BTG)로 또 다른 하드포크가 발생했습니다.Keyboard interruption in main thread... closing server.
